In [1]:
from huggingface_hub import login
login()

### Prepare Data 
* if you don't use GPT to prepare data, you could skip this part

In [3]:
import os
os.environ["OPENAI_API_KEY"] = "your_api_key"
from yogahub.data.llm.prepare_train_test_data_by_llm import process_questions

In [6]:
with open("../yogahub/data/llm/beginner_yoga_questions.txt", "r") as file:
    questions = [line.strip() for line in file]
questions[:5]

['What is yoga?',
 'What are the benefits of practicing yoga?',
 'How often should I practice yoga?',
 'What is the best time of day to practice yoga?',
 'Do I need to be flexible to start practicing yoga?']

In [7]:
default_instruction = ["Now you are my enthusiastic yoga master. Please respond with energy and positivity as a yoga teacher to the following input.",
                       "You are now my energetic yoga coach. Kindly guide me through the following query with enthusiasm and encouragement.",
                       "Assume the role of my lively yoga instructor and provide a detailed, energetic response to the input.",
                       "Imagine you are my vibrant yoga trainer. Answer the following question with enthusiasm and positivity.",
                       "As my dynamic yoga teacher, please help me with the following information with energy and encouragement.",]

process_questions(questions[:5], default_instruction, storage_folder="../datasets")

100%|██████████| 5/5 [00:22<00:00,  4.46s/it]


In [4]:
##### The pre-prepared datasets is here
import json
with open("../yogahub/data/llm/train_instruct.json", "r") as file:
    train_data = json.load(file)

In [5]:
train_data

[{'instruction': 'Imagine you are my vibrant yoga trainer. Answer the following question with enthusiasm and positivity.',
  'input': 'What is yoga?',
  'output': "Hey there! Yoga is an incredible journey—a vibrant fusion of mind, body, and spirit! It's not just a workout; it's a work-in! Through mindful movement, breath control, and meditation, yoga helps us connect deeply with ourselves and the world around us. Feel the energy flow, embrace the tranquility, and watch your strength and flexibility soar! 🌟 Ready to roll out the mat and discover your inner glow? Let’s do this! 💪✨🧘\u200d♂️"},
 {'instruction': 'Imagine you are my vibrant yoga trainer. Answer the following question with enthusiasm and positivity.',
  'input': 'What are the benefits of practicing yoga?',
  'output': "Absolutely, I'd love to share the amazing benefits of practicing yoga! 🌟 First off, yoga is fantastic for enhancing your flexibility and strength, which helps your body perform better in all aspects of life. Bu

### Download Llamav3 instruct weight

In [ ]:
from huggingface_hub import snapshot_download
# 8B-Instruct instruct version
model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
save_directory = "/tmp/meta_llama_3_8B_instruct"
# Download the entire model repository
snapshot_download(repo_id=model_name, local_dir=save_directory, token="your_token")

## LoRA Tuning Config

In [8]:
import yaml
with open("../yogahub/cfg/train/8B_lora.yaml", "r") as file:
    config = yaml.load(file, Loader=yaml.FullLoader)

In [9]:
config

{'tokenizer': {'_component_': 'torchtune.models.llama3.llama3_tokenizer',
  'path': '/tmp/meta_llama_3_8B_instruct/original/tokenizer.model'},
 'model': {'_component_': 'torchtune.models.llama3.lora_llama3_8b',
  'lora_attn_modules': ['q_proj', 'v_proj'],
  'apply_lora_to_mlp': False,
  'apply_lora_to_output': False,
  'lora_rank': 8,
  'lora_alpha': 16},
 'checkpointer': {'_component_': 'torchtune.utils.FullModelHFCheckpointer',
  'checkpoint_dir': '/tmp/meta_llama_3_8B_instruct/',
  'checkpoint_files': ['model-00001-of-00004.safetensors',
   'model-00002-of-00004.safetensors',
   'model-00003-of-00004.safetensors',
   'model-00004-of-00004.safetensors'],
  'adapter_checkpoint': None,
  'recipe_checkpoint': None,
  'output_dir': '/tmp/meta_llama_3_8B_instruct_finetune/',
  'model_type': 'LLAMA3'},
 'resume_from_checkpoint': False,
 'dataset': {'_component_': 'torchtune.datasets.instruct_dataset',
  'source': 'json',
  'data_files': 'yogahub/data/llm/train_instruct.json',
  'template':

## Fine-tune with LoRA

In [10]:
# 2 GPU
# you might have to update data_files in ../yogahub/cfg/train/8B_lora.yaml
!tune run --nproc_per_node 2 lora_finetune_distributed --config ../yogahub/cfg/train/8B_lora.yaml

Running with torchrun...
W0528 04:20:14.562000 139711112315904 torch/distributed/run.py:757] 
W0528 04:20:14.562000 139711112315904 torch/distributed/run.py:757] *****************************************
W0528 04:20:14.562000 139711112315904 torch/distributed/run.py:757] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
W0528 04:20:14.562000 139711112315904 torch/distributed/run.py:757] *****************************************
INFO:torchtune.utils.logging:Running LoRAFinetuneRecipeDistributed with resolved config:

batch_size: 2
checkpointer:
  _component_: torchtune.utils.FullModelHFCheckpointer
  adapter_checkpoint: null
  checkpoint_dir: /tmp/meta_llama_3_8B_instruct/
  checkpoint_files:
  - model-00001-of-00004.safetensors
  - model-00002-of-00004.safetensors
  - model-00003-of-00004.safetensors
  - model-00004-of-00004.saf

## Inference

In [1]:
from yogahub.pipeline.llm.basic_chat import default_pipeline, generate_query
from yogahub.models.llm.huggingface_model import load_llm_model

/mnt/lustre/home/jeff_yang/miniconda/envs/yoga/lib/python3.10/site-packages/albumentations/augmentations/geometric/transforms.py:1071: UserWarning: This augmenter is very slow. Try to use ``ElasticTransformation`` instead, which is at least 10x faster.
  warn("This augmenter is very slow. Try to use ``ElasticTransformation`` instead, which is at least 10x faster.")


In [2]:
model_id = "/tmp/meta_llama_3_8B_instruct/"
quantization = False  # Set this to False if quantization is not needed
device = "auto"  # Set this to 'cpu' or 'cuda' if specific device is needed
model, tokenizer = load_llm_model(model_id, quantization, device, peft_model=True, peft_folder="/tmp/meta_llama_3_8B_instruct_finetune/")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

INFO:yogahub.models.llm.huggingface_model:Loaded PEFT model from /tmp/meta_llama_3_8B_instruct_finetune/
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
sequence = default_pipeline(model, tokenizer)
output = generate_query("How often should I practice yoga?", sequence)

The model 'PeftModel' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'Musicg

In [11]:
print(output)

Namaste! As a yoga enthusiast, it's great that you're eager to establish a consistent practice. The frequency of yoga practice depends on your goals, schedule, and current level of physical activity. Here are some general guidelines:

* Beginners: Start with 2-3 times a week, focusing on building a strong foundation and understanding the basics.
* General wellness: Aim for 3-4 times a week, incorporating a mix of physical postures, breathing techniques, and meditation.
* Advanced practitioners: You can practice 4-5 times a week, exploring more complex postures, and refining your technique.

Remember, consistency is key. Even 10-15 minutes a day can be beneficial. Listen to your body and adjust your practice accordingly. Most importantly, make yoga a habit, and you'll start to notice the benefits in your overall well-being.

How's that? Would you like me to elaborate on any of these points or provide more guidance?
